<a href="https://colab.research.google.com/github/Moh-Bxd/Brain_tumor_detection/blob/main/Brain_tumor_segmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Getting Started

In [ ]:
!python -c "import monai" || pip install -q "monai-weekly[gdown, nibabel, tqdm, ignite]"
!python -c "import matplotlib" || pip install -q matplotlib
%matplotlib inline

! pip install kaggle
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json


In [4]:
! kaggle datasets download nikhilroxtomar/brain-tumor-segmentation
! unzip ./brain-tumor-segmentation

brain-tumor-segmentation.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  ./brain-tumor-segmentation.zip
replace images/1.png? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

# Import packages


In [5]:
import os
import cv2
import numpy as np
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import numpy as np


# Setup Data Dir

# importing dataset from kaggle


In [6]:
image_path = "./images"
mask_path = "./masks"

def load_data(image_path, mask_path, target_size=(128, 128)):
    images = []
    masks = []

    for img_name in os.listdir(image_path):
        img = cv2.imread(os.path.join(image_path, img_name))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        # Resize image to target size
        img = cv2.resize(img, target_size)

        images.append(img)

        mask = cv2.imread(os.path.join(mask_path, img_name), cv2.IMREAD_GRAYSCALE)

        # Resize mask to target size
        mask = cv2.resize(mask, target_size, interpolation=cv2.INTER_NEAREST)

        mask = np.expand_dims(mask , axis=-1)

        masks.append(mask)

    return np.array(images), np.array(masks)

images, masks = load_data(image_path, mask_path)
print(images.shape)
print(masks.shape)

(3064, 128, 128, 3)
(3064, 128, 128, 1)


# Spliting data

In [7]:
train_images , temp_images , train_labels , temp_labels = train_test_split(images, masks, test_size=0.3, random_state=42)
test_images , val_images , test_labels , val_labels = train_test_split(temp_images, temp_labels , test_size=0.3, random_state=42)


print(train_images.shape)
print(train_labels.shape)

(2144, 128, 128, 3)
(2144, 128, 128, 1)


# convert everything to tensors


In [8]:
train_images_tensor = tf.convert_to_tensor(train_images)
test_images_tensor = tf.convert_to_tensor(test_images)
val_images_tensor = tf.convert_to_tensor(val_images)

train_labels_tensor = tf.convert_to_tensor(train_labels)
test_labels_tensor = tf.convert_to_tensor(test_labels)
val_labels_tensor = tf.convert_to_tensor(val_labels)

print(train_images_tensor.shape)
print(train_labels_tensor.shape)

(2144, 128, 128, 3)
(2144, 128, 128, 1)


# resize the data

In [9]:
#input image and input mask must be a tensor
def resize(input_images, input_masks):
  for input_image , input_mask in zip(input_images, input_masks):

    if input_image.ndim == 2:
      input_image = tf.expand_dims(input_image, axis=-1)
    if input_mask.ndim == 2:
      input_mask = tf.expand_dims(input_mask, axis=-1)


    input_image = tf.image.resize(input_image, (128, 128), method="nearest")
    input_mask = tf.image.resize(input_mask, (128, 128), method="nearest")


  return input_images, input_masks


In [10]:
train_images , train_labels = resize(train_images_tensor,train_labels_tensor)
test_images , test_labels = resize(test_images_tensor,test_labels_tensor)
val_images , val_labels = resize(val_images_tensor,val_labels_tensor)


# Augment the dataset

In [11]:
def augment(input_images, input_masks):
   if tf.random.uniform(()) > 0.5:
       # Random flipping of the image and mask
       for input_image , input_mask  in zip(input_images, input_masks):
        if input_image.ndim == 2:
            input_image = tf.expand_dims(input_image, axis=-1)
        if input_mask.ndim == 2:
            input_mask = tf.expand_dims(input_mask, axis=-1)

        input_image = tf.image.flip_left_right(input_image)
        input_mask = tf.image.flip_left_right(input_mask)
   return input_images, input_masks

In [12]:
#augment only the training data
train_images , train_labels = augment(train_images,train_labels)

print(train_images.shape)
print(train_labels.shape)

(2144, 128, 128, 3)
(2144, 128, 128, 1)


# normalize data

In [13]:
def normalize(input_images, input_masks):
  for input_image , input_mask in zip(input_images, input_masks):


    input_image = tf.cast(input_image, tf.float32) / 255.0
    input_mask -= 1
  return input_images, input_masks

In [14]:
train_images , train_labels = normalize(train_images,train_labels)
test_images , test_labels = normalize(test_images,test_labels)
val_images , val_labels = normalize(val_images,val_labels)

print(train_images.shape)
print(train_labels.shape)

(2144, 128, 128, 3)
(2144, 128, 128, 1)


# Building u-net Block

In [15]:
def double_conv_block(x, n_filters):
   # Conv2D then ReLU activation
   x = layers.Conv2D(n_filters, 3, padding = "same", activation = "relu", kernel_initializer = "he_normal")(x)
   # Conv2D then ReLU activation
   x = layers.Conv2D(n_filters, 3, padding = "same", activation = "relu", kernel_initializer = "he_normal")(x)
   return x

In [16]:
def downsample_block(x, n_filters):
   f = double_conv_block(x, n_filters)
   p = layers.MaxPool2D(2)(f)
   p = layers.Dropout(0.3)(p)
   return f, p

In [17]:
def upsample_block(x, conv_features, n_filters):
   # upsample
   x = layers.Conv2DTranspose(n_filters, 3, 2, padding="same")(x)
   # concatenate
   x = layers.concatenate([x, conv_features])
   # dropout
   x = layers.Dropout(0.3)(x)
   # Conv2D twice with ReLU activation
   x = double_conv_block(x, n_filters)
   return x

# creating the u-net builder

In [18]:
def build_unet_model():
  inputs = layers.Input(shape=(128,128,3))
   # encoder: contracting path - downsample
   # 1 - downsample
  f1, p1 = downsample_block(inputs, 64)
   # 2 - downsample
  f2, p2 = downsample_block(p1, 128)
   # 3 - downsample
  f3, p3 = downsample_block(p2, 256)
   # 4 - downsample
  f4, p4 = downsample_block(p3, 512)
   # 5 - bottleneck
  bottleneck = double_conv_block(p4, 1024)
   # decoder: expanding path - upsample
   # 6 - upsample
  u6 = upsample_block(bottleneck, f4, 512)
   # 7 - upsample
  u7 = upsample_block(u6, f3, 256)
   # 8 - upsample
  u8 = upsample_block(u7, f2, 128)
   # 9 - upsample
  u9 = upsample_block(u8, f1, 64)
   # outputs
  outputs = layers.Conv2D(3, 1, padding="same", activation = "softmax")(u9)
   # unet model with Keras Functional API
  unet_model = tf.keras.Model(inputs, outputs, name="U-Net")
  return unet_model

In [19]:
unet_model = build_unet_model()

In [ ]:
unet_model.summary()

# PLoting some Images

In [ ]:
import matplotlib.pyplot as plt

plt.imshow(val_labels[78])

# Training

In [36]:
unet_model.compile(optimizer=tf.keras.optimizers.Adam(),
                  loss="sparse_categorical_crossentropy",
                  metrics="accuracy")

print(train_labels.shape)
print(test_labels.shape)
import matplotlib.pyplot as plt



(2144, 128, 128, 1)
(644, 128, 128, 1)


In [ ]:
unet_model.fit(
    x=train_images,
    y=train_labels,
    batch_size=128,
    epochs=20,
    verbose='auto',
    callbacks=None,
    validation_split=0.0,
    validation_data=(test_images, test_labels),
    shuffle=True,
    class_weight=None,
    sample_weight=None,
    initial_epoch=0,
    steps_per_epoch=None,
    validation_steps=None,
    validation_batch_size=None,
    validation_freq=1,
    max_queue_size=10,
    workers=1,
    use_multiprocessing=False
)

Epoch 1/20
16/17 [===========================>..] - ETA: 1s - loss: nan - accuracy: 0.9323